In [2]:
import pandas as pd

stock = pd.read_csv('../../data/explore/stock_url_list.csv')
stock.head()

,Stock,ราคาล่าสุด(Home),Sector,กรรมการบริษัท,งบการเงิน,บริษัท/หลักทรัพย์,ผู้ถือหุ้นรายใหญ่,ราคาย้อนหลัง
0,CHOTI,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...,/C13_FastQuote_Main.jsp?txtSymbol=CHOTI&ssoPag...
1,EE,/C13_FastQuote_Main.jsp?txtSymbol=EE,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...,/C13_FastQuote_Main.jsp?txtSymbol=EE&ssoPageId...
2,GFPT,/C13_FastQuote_Main.jsp?txtSymbol=GFPT,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...,/C13_FastQuote_Main.jsp?txtSymbol=GFPT&ssoPage...
3,LEE,/C13_FastQuote_Main.jsp?txtSymbol=LEE,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageI...
4,STA,/C13_FastQuote_Main.jsp?txtSymbol=STA,AGRI,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...,/C13_FastQuote_Main.jsp?txtSymbol=STA&ssoPageI...


# งบดุล

$$\text{Asset} = \text{Equity} + \text{Debt}$$


ข้อมูลที่ต้องการได้แก่ (กรณีดึงจาก settrade.com)

### งบดุล
- หนี้สินรวม (2017,2016,...) - Home
- เจ้าหนี้การค้าและเจ้าหนี้อื่น (2017,2016,...) - งบเต็ม
- ส่วนของผู้ถือหุ้น (2017,2016,...) - Home
- หนี่สินที่มีภาระดอกเบี้ย (2017,2016,...) - งบเต็ม ซึ่งประกอบไปด้วย
    - เงินเบิกเกินบัญชีและเงินกู้ยืมระยะสั้นจากสถาบันการเงิน(งบเต็ม)
    - ส่วนของหนี้สินระยะยาวที่ถึงกำหนดชำระภายในหนึ่งปี(งบเต็ม) : อาจจะสนแค่ เงินกู้ยืมระยะยาว-สถาบันการเงิน+ตราสารหนี้
    - หนี้สินระยะยาว-สุทธิจากส่วนที่ถึงกำหนดชำระภายในหนึ่งปี(งบเต็ม) : อาจจะสนแค่ เงินกู้ยืมระยะยาว-สถาบันการเงิน+ตราสารหนี้
    
### ตรวจสอบส่วนของผู้ถือหุ้นว่ามาจากกำไรหรือระดมทุนเพิ่ม
- มูลค่าหุ้นที่เรียกชำระแล้ว (2017,2016,2015,...) - Home

ข้อมูลจากตาราง stock ทำให้เรารู้ url ของข้อมูลที่เราจะไปดึงมาแล้ว ดังนั้นทำฟังก์ชั่นสำหรับดึงข้อมูลงบการเงิน จะได้

In [315]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import re

In [316]:
def is_float(input):
    try:
        num = float(input)
    except ValueError:
        return False
    return True

In [317]:
def getStockDetail(url,stock):
    page = requests.get(url) # ดูดหน้าเว็บมา
    
    if page.status_code == 200 :
        # All HTML of Page
        soup = BeautifulSoup(page.content, "html5lib")
        # ซอยข้อมูลทั้ง page เป็นทีละส่วนๆ แล้วลงลึกไปเอาข้อมูลทีละจุดๆ
        html = list(soup.children)[1]
        head = list(html.children)[0]
        body = list(html.children)[2]
        
        # ***
        table_info = list(body.find(class_='table-info'))
        table_header1 = table_info[3]
        table_body1 = table_info[5]
        table_body_row1 = table_body1.find_all('tr')
        table_header2 = table_info[6]
        table_body2 = table_info[8]

        # Date
        financial_head = table_header1.find_all('strong') # งวดงบการเงิน ณ วันที่
        financial_date_html = financial_head[1:len(financial_head)]
        date_text = [ list(date_html.stripped_strings) for date_html in financial_date_html ]
        dates = [ year_date[1] for year_date in date_text if len(year_date) > 0 ]
        
        # Asset
        asset_row = table_body_row1[1].find_all('td') # สินทรัพย์รวม
        asset_list = [ list(asset.stripped_strings) for asset in asset_row if len(asset) > 0 ]
        asset_text = [ asset[0].replace(',','') for asset in asset_list if len(asset) > 0 ]
        assets = [ float(asset) for asset in asset_text if is_float(asset) ]
        
#         previous_price_url = nav_menu[3].find('a').get('href')
#         company_stock_name = nav_menu[5].find('a').text #บริษัท/หลักทรัพย์
#         company_stock_url = nav_menu[5].find('a').get('href') 
#         board_of_director_name = nav_menu[7].find('a').text #กรรมการบริษัท
#         board_of_director_url = nav_menu[7].find('a').get('href') 
#         major_share_holder_name = nav_menu[9].find('a').text #ผู้ถือหุ้นรายใหญ่
#         major_share_holder_url = nav_menu[9].find('a').get('href')  
#         financial_name = nav_menu[11].find('a').text #งบการเงิน
#         financial_url = nav_menu[11].find('a').get('href')

#         # Convert to DataFrame         
#         stock_df = pd.DataFrame({previous_price_name: [previous_price_url], 
#                                  company_stock_name: [company_stock_url],
#                                  board_of_director_name: [board_of_director_url],
#                                  major_share_holder_name: [major_share_holder_url],
#                                  financial_name: [financial_url]
#                                 })
#         stock_df['Stock'] = stock
        return assets
    else:
        print("Can't get content from this URL!!!")

In [318]:
prefix_url = 'https://www.settrade.com'
suffix_url = '/C13_FastQuote_Main.jsp?txtSymbol=LEE&ssoPageId=13&selectPage=6'
settrade_url = prefix_url + suffix_url

In [319]:
xx = getStockDetail(settrade_url,'LEE')
xx

[2887.18, 2827.41, 2995.53, 2989.21]

In [175]:
len(xx[0])

10